In [1]:
# Welcome to RAD-MMM inference tutorial

## imports
import pytorch_lightning as pl
import sys
import yaml
sys.path.append('/akshit/scratch/RAD-MMM/vocoders')
sys.path.append('/akshit/scratch/RAD-MMM')
from pytorch_lightning.cli import LightningCLI
from tts_lightning_modules import TTSModel
from data_modules import BaseAudioDataModule
from jsonargparse import lazy_instance
from decoders import RADMMMFlow
from loss import RADTTSLoss
import inspect
from pytorch_lightning.callbacks import ModelCheckpoint
from training_callbacks import LogDecoderSamplesCallback, \
    LogAttributeSamplesCallback
from utils import get_class_args
from tts_text_processing.text_processing import TextProcessing
from common import Encoder

In [2]:
from tts_main import RADTTSLightningCLI,lcli

In [3]:
pl.__version__

'2.0.2'

In [4]:
# Set paths
attribute_model_path = "/akshit/scratch/generator_ckpt/radmmm_public/attribute_model.ckpt"
decoder_model_path = "/akshit/scratch/generator_ckpt/radmmm_public/decoder.ckpt"
gen_config_path = "/akshit/scratch/generator_ckpt/radmmm_public/config_interactive.yaml"
voc_model_path = "/akshit/scratch/generator_ckpt/hfg_public/g_00072000"
voc_config_path = "/akshit/scratch/generator_ckpt/hfg_public/config_16khz.json"
phonemizer_cfg='{"en_US": "assets/en_US_word_ipa_map.txt","es_MX": "assets/es_MX_word_ipa_map.txt","de_DE": "assets/de_DE_word_ipa_map.txt","en_UK": "assets/en_UK_word_ipa_map.txt","es_CO": "assets/es_CO_word_ipa_map.txt","es_ES": "assets/es_ES_word_ipa_map.txt","fr_FR": "assets/fr_FR_word_ipa_map.txt","hi_HI": "assets/hi_HI_word_ipa_map.txt","pt_BR": "assets/pt_BR_word_ipa_map.txt","te_TE": "assets/te_TE_word_ipa_map.txt"}'

In [5]:
# import mock

# with mock.patch("sys.argv", ["predict", "--config="+gen_config_path,\
#                              # "--ckpt_path="+attribute_model_path,\
#                              "--model.predict_mode=tts", \
#                              "--data.inference_transcript=model_inputs/resynthesis_prompts.json", \
#                              "--model.prediction_output_dir=/akshit/scratch/RAD-MMM/tutorials/out1", \
#                              "--trainer.devices=1", "--data.batch_size=1", \
#                              "--model.vocoder_checkpoint_path="+voc_model_path, \
#                              "--model.vocoder_config_path="+voc_config_path, \
#                              "--data.phonemizer_cfg="+'{"en_US": "assets/en_US_word_ipa_map.txt","es_MX": "assets/es_MX_word_ipa_map.txt","de_DE": "assets/de_DE_word_ipa_map.txt","en_UK": "assets/en_UK_word_ipa_map.txt","es_CO": "assets/es_CO_word_ipa_map.txt","es_ES": "assets/es_ES_word_ipa_map.txt","fr_FR": "assets/fr_FR_word_ipa_map.txt","hi_HI": "assets/hi_HI_word_ipa_map.txt","pt_BR": "assets/pt_BR_word_ipa_map.txt","te_TE": "assets/te_TE_word_ipa_map.txt"}', \
#                              "--model.encoders_path="+decoder_model_path, \
#                              "--model.decoder_path="+decoder_model_path, \
#                              "--model.output_directory=/akshit/scratch/RAD-MMM/tutorials/run1"]):
#     cli = RADTTSLightningCLI(TTSModel, BaseAudioDataModule, save_config_kwargs={"overwrite": True}, run=False)

In [6]:
# results in issue - SIGSEV

# cli.trainer.predict(model=cli.model,datamodule=cli.datamodule)

In [7]:
# attempting to load the model
import pytorch_lightning
import inspect
import torch
from decoders import RADMMMFlow
# inspect.signature(pytorch_lightning.cli.instantiate_module)
inspect.signature(torch.nn.Module.load_state_dict)

<Signature (self, state_dict: Mapping[str, Any], strict: bool = True)>

In [8]:
with open("/akshit/scratch/RAD-MMM/tutorials/run1/lightning_logs/version_12/hparams.yaml", "r") as f:
    hparams = yaml.safe_load(f)

with open(gen_config_path, "r") as f:
    gen_config = yaml.safe_load(f)
# hparams['decoder']['init_args']

In [9]:
# https://github.com/mauvilsa/lightning/blob/c71406f390b89be7bbe3415e57dd785095675779/src/lightning/pytorch/cli.py
def instantiate_class(init):
    """Instantiates a class with the given args and init.

    Args:
        args: Positional arguments required for instantiation.
        init: Dict of the form {"class_path":...,"init_args":...}.

    Returns:
        The instantiated class object.
    """
    kwargs = init.get("init_args", {})
    class_module, class_name = init["class_path"].rsplit(".", 1)
    module = __import__(class_module, fromlist=[class_name])
    args_class = getattr(module, class_name)
    return args_class(**kwargs)
    
# https://lightning.ai/forums/t/best-way-to-use-load-from-checkpoint-when-model-contains-other-models/2094

In [10]:
ttsmodel_kwargs={}
for k,v in hparams.items():
    if type(v) == dict and 'class_path' in v:
        print(k)
        ttsmodel_kwargs[k] = instantiate_class(v)
    elif k != "_instantiator":
        ttsmodel_kwargs[k] = v

decoder


/akshit/scratch/RAD-MMM/common.py:557: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2425.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/opt/conda/lib/python3.10/site-packages/torch/functional.py:1728: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_p

decoder_loss
text_encoder
Applying spectral norm to text encoder LSTM
f0_predictor
f0_predictor_loss
energy_predictor
energy_predictor_loss
voiced_predictor
voiced_predictor_loss
duration_predictor
duration_predictor_loss
speaker_embed_regularization_loss
speaker_accent_cross_regularization_loss


In [11]:
ttsmodel_kwargs

{'decoder': RADMMMFlow(
   (length_regulator): LengthRegulator()
   (context_lstm): LSTM(1060, 528, batch_first=True, bidirectional=True)
   (flows): ModuleList(
     (0): FlowStep(
       (invtbl_conv): DataInitializedInvertible1x1Conv()
       (coupling_tfn): AffineTransformationLayer(
         (affine_param_predictor): WN(
           (in_layers): ModuleList(
             (0): ConvNorm(
               (conv): PartialConv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
             )
             (1): ConvNorm(
               (conv): PartialConv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(4,), dilation=(2,))
             )
             (2): ConvNorm(
               (conv): PartialConv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(8,), dilation=(4,))
             )
             (3): ConvNorm(
               (conv): PartialConv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(16,), dilation=(8,))
             )
           )
           (res_skip_layers):

In [12]:
# model2=TTSModel(**ttsmodel_kwargs)

In [13]:
gen_config["data"]["inference_transcript"]="model_inputs/resynthesis_prompts.json" #ToDo
gen_config["data"]["batch_size"]=1
gen_config["data"]["phonemizer_cfg"]=phonemizer_cfg

In [14]:
phonemizer_cfg

'{"en_US": "assets/en_US_word_ipa_map.txt","es_MX": "assets/es_MX_word_ipa_map.txt","de_DE": "assets/de_DE_word_ipa_map.txt","en_UK": "assets/en_UK_word_ipa_map.txt","es_CO": "assets/es_CO_word_ipa_map.txt","es_ES": "assets/es_ES_word_ipa_map.txt","fr_FR": "assets/fr_FR_word_ipa_map.txt","hi_HI": "assets/hi_HI_word_ipa_map.txt","pt_BR": "assets/pt_BR_word_ipa_map.txt","te_TE": "assets/te_TE_word_ipa_map.txt"}'

In [15]:

model2 = TTSModel.load_from_checkpoint(checkpoint_path=attribute_model_path,\
                                      **ttsmodel_kwargs)


loading:  assets/en_US_word_ipa_map.txt
loading:  assets/es_MX_word_ipa_map.txt
loading:  assets/de_DE_word_ipa_map.txt
loading:  assets/en_UK_word_ipa_map.txt
loading:  assets/es_CO_word_ipa_map.txt
loading:  assets/es_ES_word_ipa_map.txt
loading:  assets/fr_FR_word_ipa_map.txt
loading:  assets/hi_HI_word_ipa_map.txt
loading:  assets/pt_BR_word_ipa_map.txt
loading:  assets/te_TE_word_ipa_map.txt
Number of symbols: 439
updating the speakers set: 7
Initializing f0 predictor
ConvLSTMLinearDAP(
  (bottleneck_layer): BottleneckLayer(
    (projection_fn): ConvNorm(
      (conv): Conv1d(520, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (non_linearity): LeakyReLU(negative_slope=0.01)
  )
  (feat_pred_fn): ConvLSTMLinear(
    (dropout): Dropout(p=0.5, inplace=False)
    (convolutions): ModuleList(
      (0): ConvNorm(
        (conv): Conv1d(48, 256, kernel_size=(15,), stride=(1,), padding=(7,))
      )
      (1-2): 2 x ConvNorm(
        (conv): Conv1d(256, 256, kernel_size=(15,),

In [16]:
data_module = BaseAudioDataModule(**gen_config['data'])

loading:  assets/en_US_word_ipa_map.txt
loading:  assets/es_MX_word_ipa_map.txt
loading:  assets/de_DE_word_ipa_map.txt
loading:  assets/en_UK_word_ipa_map.txt
loading:  assets/es_CO_word_ipa_map.txt
loading:  assets/es_ES_word_ipa_map.txt
loading:  assets/fr_FR_word_ipa_map.txt
loading:  assets/hi_HI_word_ipa_map.txt
loading:  assets/pt_BR_word_ipa_map.txt
loading:  assets/te_TE_word_ipa_map.txt
Number of symbols: 439


In [17]:
data_module.setup("predict")
# dm.teardown(stage="predict")

{'basedir': '/home/dcg-adlr-rbadlani-data/multilingual-dataset/opensource/LJSpeech-1.0', 'sampling_rate': '16khz', 'filelist_basedir': 'datasets/opensource/', 'filelist': 'LJSpeech/ljs_audiopath_text_sid_emotion_duration_train_filelist_phonemized.txt', 'language': 'en_US', 'phonemized': True}
processing file: datasets/opensource/LJSpeech/ljs_audiopath_text_sid_emotion_duration_train_filelist_phonemized.txt
{'basedir': '/home/dcg-adlr-rbadlani-data/multilingual-dataset/opensource/HUI-Audio-Corpus-German/Bernd_Ungerer', 'sampling_rate': '16khz', 'filelist_basedir': 'datasets/opensource/', 'filelist': 'HUI-Audio-Corpus-German/Bernd_Ungerer/berndungerer_audiopath_text_sid_emotion_duration_train_filelist_filtered_phonemized.txt', 'language': 'de_DE', 'phonemized': True}
processing file: datasets/opensource/HUI-Audio-Corpus-German/Bernd_Ungerer/berndungerer_audiopath_text_sid_emotion_duration_train_filelist_filtered_phonemized.txt
{'basedir': '/home/dcg-adlr-rbadlani-data/multilingual-datase

In [41]:
data_module.predictset.cuda()

AttributeError: 'TextOnlyData' object has no attribute 'cuda'

In [19]:
# model2.tp_inference.phonemizer_backend_dict
data_module.tp.phonemizer_backend_dict

{'en_US': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f00c8059570>,
 'es_MX': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f022a5cd570>,
 'de_DE': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f022a5cceb0>,
 'en_UK': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f022a5cd030>,
 'es_CO': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f022a5ccdc0>,
 'es_ES': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f022a5cd240>,
 'fr_FR': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f00c80588e0>,
 'hi_HI': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f00c805bf40>,
 'pt_BR': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f00c805b760>,
 'te_TE': <tts_text_processing.grapheme_dictionary.Grapheme2PhonemeDictionary at 0x7f00c805b400>}

In [27]:
a_dl = data_module.predict_dataloader()

In [38]:
model2.cuda()

TTSModel(
  (text_embeddings): Embedding(426, 512)
  (text_encoder): Encoder(
    (convolutions): ModuleList(
      (0-2): 3 x Sequential(
        (0): ConvNorm(
          (conv): PartialConv1d(520, 520, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): InstanceNorm1d(520, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      )
    )
    (lstm): LSTM(520, 260, batch_first=True, bidirectional=True)
  )
  (speaker_embeddings): Embedding(21, 16)
  (accent_embeddings): Embedding(7, 8)
  (attention): ConvAttention(
    (softmax): Softmax(dim=3)
    (log_softmax): LogSoftmax(dim=3)
    (key_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(528, 1056, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(1056, 80, kernel_size=(1,), stride=(1,))
      )
    )
    (query_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(80, 160, kernel_size=(3,), stride=(1,), padding=(1,))
 

In [42]:
model2.forward(next(iter(a_dl)))

{d ɛ ɾ} {v eː ts ˈiː ɾ} {n ˈɑː m} { ˈa l ə s,} {v ˈa s} { iː m} {d ɛ ɾ} {k ˈøː n ɪ ç} {ɡ ˈɑː p,} {d ?? ç v ˈa n d ɜ t ə} {d iː} {v ˈy s t ə} { ɪ n} {d ɛ ɾ} {l ˈɛ ŋ ə} { ʊ n t} { ɪ n} {d ɛ ɾ} {b ɾ ˈaɪ t ə,} {b ɪ s} { ɛ ɾ} { ɪ n} {d a s} {l ˈa n t} {d ɛ s} {k ˈøː n ɪ ç s} {ʃ ˈɑ m ɛ ç} {k ˌɑː m.} is NOT phonemized...
de_DE
de_DE|{d ɛ ɾ} {v eː ts ˈiː ɾ} {n ˈɑː m}{ ˈa l ə s,} {v ˈa s}{ iː m} {d ɛ ɾ} {k ˈøː n ɪ ç} {ɡ ˈɑː p,} {d ?? ç v ˈa n d ɜ t ə} {d iː} {v ˈy s t ə}{ ɪ n} {d ɛ ɾ} {l ˈɛ ŋ ə}{ ʊ n t}{ ɪ n} {d ɛ ɾ} {b ɾ ˈaɪ t ə,} {b ɪ s}{ ɛ ɾ}{ ɪ n} {d a s} {l ˈa n t} {d ɛ s} {k ˈøː n ɪ ç s} {ʃ ˈɑ m ɛ ç} {k ˌɑː m.}
de_DE|[51, 178, 252, 0, 131, 64, 304, 114, 302, 71, 304, 252, 0, 88, 302, 164, 304, 86, 302, 40, 85, 176, 111, 25, 0, 131, 302, 40, 111, 71, 304, 86, 0, 51, 178, 252, 0, 73, 302, 150, 304, 88, 214, 146, 0, 187, 302, 164, 304, 95, 25, 0, 51, 31, 31, 146, 131, 302, 40, 88, 51, 179, 113, 176, 0, 51, 71, 304, 0, 131, 302, 135, 111, 113, 176, 214, 88, 0, 51, 178, 252, 0, 85, 302, 178, 1

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
resyn_input.keys()

In [ ]:
## some runtime error
with open("/akshit/scratch/RAD-MMM/tutorials/run1/lightning_logs/version_12/hparams.yaml", "r") as f:
    hparams = yaml.safe_load(f)
    
# model2 = pytorch_lightning.cli.instantiate_module(TTSModel,config=hparams)


In [ ]:
import torch
model3=torch.load(attribute_model_path)

In [ ]:
model3

In [ ]:
model4 = torch.load(decoder_model_path)

In [ ]:
model4

In [ ]:
# run=True, like cmd line; https://lightning.ai/docs/pytorch/stable/cli/lightning_cli_advanced_3.html, fails with SIGSEGV
# inp = ["predict", "--config="+gen_config_path,\
#          # "--ckpt_path="+attribute_model_path,\
#          "--model.predict_mode=tts", \
#          "--data.inference_transcript='model_inputs/resynthesis_prompts.json'", \
#          "--model.prediction_output_dir='/akshit/scratch/RAD-MMM/tutorials/out1'", \
#          "--trainer.devices=1", "--data.batch_size=1", \
#          "--model.vocoder_checkpoint_path="+voc_model_path, \
#          "--model.vocoder_config_path="+voc_config_path, \
#          "--data.phonemizer_cfg="+'{"en_US": "assets/en_US_word_ipa_map.txt","es_MX": "assets/es_MX_word_ipa_map.txt","de_DE": "assets/de_DE_word_ipa_map.txt","en_UK": "assets/en_UK_word_ipa_map.txt","es_CO": "assets/es_CO_word_ipa_map.txt","es_ES": "assets/es_ES_word_ipa_map.txt","fr_FR": "assets/fr_FR_word_ipa_map.txt","hi_HI": "assets/hi_HI_word_ipa_map.txt","pt_BR": "assets/pt_BR_word_ipa_map.txt","te_TE": "assets/te_TE_word_ipa_map.txt"}', \
#          "--model.encoders_path="+decoder_model_path, \
#          "--model.decoder_path="+decoder_model_path, \
#          "--model.output_directory='/akshit/scratch/RAD-MMM/tutorials/run1'"]
# lcli(inp)

In [ ]:
# https://github.com/Lightning-AI/pytorch-lightning/pull/18105 

In [ ]:
# # load the config files

# with open(gen_config_path, "r") as f:
#     gen_config = yaml.safe_load(f)
    
#     # set config
#     gen_config["ckpt_path"] = attribute_model_path
#     gen_config["model"]["predict_mode"]="tts"
#     gen_config["model"]["prediction_output_dir"]="/akshit/scratch/RAD-MMM/tutorials/out1" #ToDo
#     gen_config["model"]["vocoder_checkpoint_path"]=voc_model_path
#     gen_config["model"]["vocoder_config_path"]=voc_model_path
#     gen_config["model"]["decoder_path"]=decoder_model_path
#     gen_config["model"]["encoders_path"]=decoder_model_path
#     gen_config["model"]["output_directory"]="/akshit/scratch/RAD-MMM/tutorials/run1"
#     gen_config["trainer"]["devices"]=1
    # gen_config["data"]["inference_transcript"]="model_inputs/resynthesis_prompts.json" #ToDo
    # gen_config["data"]["batch_size"]=1
    # gen_config["data"]["phonemizer_cfg"]=phonemizer_cfg
    
#     # set defaults
#     gen_config["checkpoint_callback"]["filename"]="latest-epoch_{epoch}-iter_{global_step:.0f}"
#     gen_config["checkpoint_callback"]["monitor"]="global_step"
#     gen_config["checkpoint_callback"]["mode"]= "max"
#     gen_config["checkpoint_callback"]["every_n_train_steps"]= 3000
#     gen_config["checkpoint_callback"]["dirpath"]= "/debug"
#     gen_config["checkpoint_callback"]["save_top_k"] = -1
#     gen_config["checkpoint_callback"]["auto_insert_metric_name"]=False
    
#     # linking args
#     gen_config["model"]["phonemizer_cfg"]=gen_config["data"]["phonemizer_cfg"]
#     gen_config["model"]["add_bos_eos_to_text"]=gen_config["data"]["add_bos_eos_to_text"]
#     gen_config["model"]["append_space_to_text"]=gen_config["data"]["append_space_to_text"]
#     gen_config["model"]["prepend_space_to_text"]=gen_config["data"]["prepend_space_to_text"]
#     gen_config["model"]["handle_phoneme_ambiguous"]=gen_config["data"]["handle_phoneme_ambiguous"]
#     gen_config["model"]["handle_phoneme"]=gen_config["data"]["handle_phoneme"]
#     gen_config["model"]["p_phoneme"]=gen_config["data"]["p_phoneme"]
#     gen_config["model"]["phoneme_dict_path"]=gen_config["data"]["phoneme_dict_path"]
#     gen_config["model"]["heteronyms_path"]=gen_config["data"]["heteronyms_path"]
#     gen_config["model"]["cleaner_names"]=gen_config["data"]["cleaner_names"]
#     gen_config["model"]["symbol_set"]=gen_config["data"]["symbol_set"]
#     gen_config["model"]["sampling_rate"]=gen_config["data"]["sampling_rate"]
#     gen_config["checkpoint_callback"]["dirpath"]=gen_config["model"]["output_directory"]
#     gen_config["trainer"]["default_root_dir"]=gen_config["model"]["output_directory"]
    
    

In [ ]:
model = cli.model
data_module = cli.datamodule
trainer = cli.trainer

In [ ]:
predictions = trainer.predict(model=model)

In [ ]:
model = config_init.model
data_module = config_init.data
trainer = config_init.trainer

In [ ]:
# first we go through one input at a time

In [ ]:
# resynthesis input
resyn_input = {
      "script": "{ʈ ˈɪ h ə ɾ ˌi} {b ˈã dʰ} {b ˈʌ n n ˌeː} {k eː} {d ɔː ɾ ˈaː n} {s ˈʌ n t oː} {n ˈeː} {ʋ ɪ ɾ ˈoː dʰ} {ɟ ə ɾ ˈuː ɾ} {k ˈɪ j aː} {l ˈeː k ɪ n} {pʰ ˈɪ ɾ} {tʰ ˈʌ m} {ɡ ˈʌ eː}",
      "spk_id": "indic-iiit-hyd-female",
      "decoder_spk_id": "indic-iiit-hyd-female",
      "duration_spk_id": "indic-iiit-hyd-female",
      "energy_spk_id": "indic-iiit-hyd-female",
      "f0_spk_id": "indic-iiit-hyd-female",
      "language": "hi_HI",
      "emotion": "other"}

In [ ]:
# here is a gradio interface to try out multiple combinations